# TPU BiTempered RAdam Cutmix CV5 tf.data TFRec

In this notebook i'll be exploring the cassav leaf disease detection dataset while studying different techniques for Deep Learning and Dataset pipeline.

In [ ]:
!pip install -U tensorflow==2.3.2 &> /dev/null
!pip install --upgrade tensorflow_hub &> /dev/null
!pip install -U tfa-nightly &> /dev/null
print("update TPU server tensorflow version...")
!pip install cloud-tpu-client &> /dev/null


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import tensorflow_hub
import tensorflow_addons
import tensorflow as tf 
import tensorflow_addons as tfa
from tensorflow_addons.optimizers import RectifiedAdam as RAdam
from tensorflow_addons.optimizers import Lookahead
from cloud_tpu_client import Client
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from kaggle_datasets import KaggleDatasets
import re
from pathlib import Path
import os
import glob
import gc
from functools import partial

In [ ]:
#Configurando TPU kaggle
Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')
gcs_path = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
#Variáveis globais
image_shape = [512,512,3]
BATCH_SIZE = 128
AUG_BATCH=BATCH_SIZE
AUTOTUNE = tf.data.experimental.AUTOTUNE
files = tf.io.gfile.glob(gcs_path+'/train_tfrecords/*.tfrec')
steps_per_epoch = int(count_data_items(files)//BATCH_SIZE)
ohe=True
EPOCHS=40

In [ ]:
IMAGE_SIZE=[512,512]

## Aumento de Dados <a name=aumento></a>
Muitas vezes, a tarefa que queremos completar é muito complexo e não temos dados suficientes para treinar nosso modelo. Neste caso, é possível aumentar a qualidade dos nossos dados realizando uma técnica chamada [Data Augmentation](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0197-0). Isso é uma das várias maneiras de mostrar que a qualidade do nosso modelo depende mais dos nossos dados que do modelo em si. Para este modelo, usaremos as seguintes técnicas para a mudança de imagens: Rotação, Shear, Zoom Horizontal e Vertical, Translação Horizontal e Vertical, e duas técnicas avançadas chamadas [Cutmix](https://sarthakforwet.medium.com/cutmix-a-new-strategy-for-data-augmentation-bbc1c3d29aab) e [Mixup](https://paperswithcode.com/method/mixup). Em resumo, Cutmix mistura duas imagens e labels recortando um pedaço de uma imagem e colando por cima de outra, e combinando os labels com proporções ao tamanho do recorte, através da fórmula:
$$
y = \lambda y_i + (1-\lambda)y_j.
$$

Mixup é similar em questão de combinar diferentes proporções de duas imagens, porém esse mistura é feita em duas imagens inteiras, através da fórmula:

$$
x = \lambda x_i + (1-\lambda)x_j \\
y = \lambda y_i + (1-\lambda)y_j.
$$
Todas os aumentos são feitas de forma aleatória, com alcance de parâmetros bem definidos. 

Tensorflow contém várias funções eficientes para o aumento de dados que podem serem usados diretos no pipeline de dados, e muitos podem até serem usados como uma camada, excelente para treino em GPUs. Neste notebook porém, não usaremos destes módulos e implementaremos de forma direta estas funções, pois muitos delas não podem ser usados para treinamento em TPU, e poderemos ter mais controle das distribuíções de valores de transformação (rotacionando aleatoriamente usando uma distribuíção normal). 

This cell defnies the functions to transform an image in random fashion. It augments the dataset applying: rotation, shearing, zooms and shifts.

In [ ]:
#Implementação dos aumentos básicos (rotação, sheering, zoom e translação)

ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = np.pi * rotation / 180.
    shear    = np.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform_mat(image, DIM=IMAGE_SIZE[0]):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d, [DIM, DIM,3])

This cell defines the function to apply [mixup](https://towardsdatascience.com/2-reasons-to-use-mixup-when-training-yor-deep-learning-models-58728f15c559).

In [ ]:
#Aqui está a implementação do mixup

def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 5
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2

This cell defines the function to apply [cutmix](https://sarthakforwet.medium.com/cutmix-a-new-strategy-for-data-augmentation-bbc1c3d29aab).

In [ ]:
#Aqui está a implementação de cutmix
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 5
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2

This function is responsible for choosing if it applies mixup or cutmix, or anything at all. It also applies random sturation, contrast, brightness and horizontal flipping.

In [ ]:
#E aqui está a função que combina todos estes passos. Nós não combinamos cutmix e mixup de uma vez, mas uma proporção SWITCH e (1-SWITCH) de vezes.  De qualquer forma,
#usaremos uma dessas técnicas 66% das vezes.
def transform(image,label):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = IMAGE_SIZE[0]
    CLASSES = 5
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image1 = []
    for j in range(AUG_BATCH):
        img = transform_mat(image[j,])
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_saturation(img, 0.7, 1.3)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        image1.append(img)
        
    image1 = tf.reshape(tf.stack(image1),(AUG_BATCH,DIM,DIM,3))
    image2, label2 = cutmix(image1, label, CUTMIX_PROB)
    image3, label3 = mixup(image1, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image4,label4

This cell defines a function that reads data from tfrecords format, decodes into a jpeg format, than decodes the image into a tensor, while rescaling it form 0-255 to 0-1. 

O tf.data permite que a gente crie um pipeline de dados para a leitura de dados em disco. Desta forma, não precisaremos de uma memória RAM enorme para o treinamento de um modelo. Porém, há a possibilidade de bottlenecks caso a contrução do pipeline não for feita de maneira correta, fazendo com que tempo seja gasto lendo e processando os dados enquanto o modelo está estagnado esperando mais dados para treinamento. Para mais informações sobre tf.data, leia [esta guia](https://www.tensorflow.org/guide/data).



In [ ]:
#Esta função define o processo de leitura de imagem. Primeiro ela decodifica o formato jpeg, depois transforma os dados em float, 
#e após isso redimensiona a imagem para o tamanho escolhido
def decode_img(img):
    image = tf.image.decode_jpeg(img, channels=3)
    image = tf.cast(image, tf.float32)/255.0
    image = tf.reshape(image, [512,512,3])
    return image
#Está função recebe um example tfrecord e retorna a imagem e label, para o caso de treinamento, ou imagem e nome de imagem, para o caso em que estamos na fase de teste.
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_img(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
#Está função transforma nossos labels em codificação númerica (1-5) para codificação one-hot (e.g. 0,0,1,0,0). Desta forma poderá ser usados as técnicas MixUp e CutMix.
def onehot(image,label):
    CLASSES = 5
    return image,tf.one_hot(label,CLASSES)

In [ ]:
#Está função recebe uma lista de caminhos para os tfrecords e cria um dataset.
def create_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset



#Dataset for unlabeled images (test set)
def inference_dataset(files,number_test_files, return_ids):
    
    ds = create_dataset(files, labeled=False, ordered=True)
    ds = ds.batch(number_test_files)
    if return_ids:
        ds = ds.map(lambda img, ids: ids)
    else:
        ds = ds.map(lambda img, ids: img)
    
    return ds

Cria um dataset com a função acima e aplica as transformações, depois embaralha as imagens, junta as imagens em batchs, e usa prefetch para a otimização de pipeline. O prefetch otimiza o pipeline processando batchs de imagens enquanto o modelo está treinando com um batch já recebido. Sem este processo, o modelo teria que esperar o cpu processar os dados toda vez que ela terminar de processar os dados recebidos. Nota-se que usamos o batch antes de aplicar o agumentação de dados, pois o cutmix e mixup precisam de um batch de dados para serem usados.

In [ ]:
def training_dataset(files):
    ds = create_dataset(files)
    ds = ds.cache()
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.map(transform, num_parallel_calls=AUTOTUNE)
    ds = ds.unbatch()
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(AUTOTUNE)
    return ds

Este conjunto de dados se aplica ao set de dados para validação. Um processo comum ao Machine Learning em geral é separar nossos dados em treino e validação. Desta forma teremos garantia de que nosso modelo está generalizando para dados fora do conjunto de treinamento, o que é o objetivo principal de machine learning. Notamos que os dados de validação não são aumentados, pois isso só atrapalharia a acurácia do modelo. Existe porém, uma técnica chamada Test Time Augmentation (TTA), onde a imagem é transformada em N diferentes levementes modificadas, para em seguida agregar a previsão de todos elas. 

In [ ]:
#Notamos que não usamos agumentação de dados. Queremos saber se nosso modelo generaliza para dados que iremos usar em prática.
def validation_dataset(files):
    ds = create_dataset(files)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.map(onehot, num_parallel_calls=AUTOTUNE)
    ds = ds.cache()
    ds = ds.prefetch(AUTOTUNE)
    
    return ds

Abaixo está a demonstração das imagens modificadas que o modelo irá receber. Podemos ver demonstrações de cutmix e mixup.

In [ ]:
ds = training_dataset(files).unbatch()





fig, axs = plt.subplots(3,3,figsize=(12,12))
for i,item in enumerate(ds.take(9)):
    axs.flat[i].imshow(item[0])
    axs.flat[i].axis('off')

In [ ]:
ds = validation_dataset(files).unbatch()





fig, axs = plt.subplots(3,3,figsize=(12,12))
for i,item in enumerate(ds.take(9)):
    axs.flat[i].imshow(item[0])
    axs.flat[i].axis('off')

In [ ]:
import shutil
shutil.copy('../input/bi-temepered-loss/loss.py', './')
from loss import bi_tempered_logistic_loss

T_1 = 0.8
T_2 = 1.2
SMOOTH_FRACTION = 0.1
N_ITER = 5

with strategy.scope():
    class BiTemperedLogisticLoss(tf.keras.losses.Loss):
        def __init__(self, t1=T_1, t2=T_2, lbl_smth=SMOOTH_FRACTION, n_iter=5):
          super(BiTemperedLogisticLoss, self).__init__()
          self.t1 = t1
          self.t2 = t2
          self.lbl_smth = lbl_smth
          self.n_iter = n_iter

        def call(self, y_true, y_pred):
          return bi_tempered_logistic_loss(y_pred, y_true, self.t1, self.t2, self.lbl_smth, self.n_iter)

In [ ]:
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
# pylint: disable=invalid-name
"""EfficientNet models for Keras.

Reference paper:
  - [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks]
    (https://arxiv.org/abs/1905.11946) (ICML 2019)
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import math
import os

from tensorflow.python.keras import backend
from tensorflow.python.keras import layers
from tensorflow.python.keras.applications import imagenet_utils
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.utils import data_utils
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export


BASE_WEIGHTS_PATH = 'https://storage.googleapis.com/keras-applications/'

WEIGHTS_HASHES = {
    'b0': ('902e53a9f72be733fc0bcb005b3ebbac',
           '50bc09e76180e00e4465e1a485ddc09d'),
    'b1': ('1d254153d4ab51201f1646940f018540',
           '74c4e6b3e1f6a1eea24c589628592432'),
    'b2': ('b15cce36ff4dcbd00b6dd88e7857a6ad',
           '111f8e2ac8aa800a7a99e3239f7bfb39'),
    'b3': ('ffd1fdc53d0ce67064dc6a9c7960ede0',
           'af6d107764bb5b1abb91932881670226'),
    'b4': ('18c95ad55216b8f92d7e70b3a046e2fc',
           'ebc24e6d6c33eaebbd558eafbeedf1ba'),
    'b5': ('ace28f2a6363774853a83a0b21b9421a',
           '38879255a25d3c92d5e44e04ae6cec6f'),
    'b6': ('165f6e37dce68623721b423839de8be5',
           '9ecce42647a20130c1f39a5d4cb75743'),
    'b7': ('8c03f828fec3ef71311cd463b6759d99',
           'cbcfe4450ddf6f3ad90b1b398090fe4a'),
}

DEFAULT_BLOCKS_ARGS = [{
    'kernel_size': 3,
    'repeats': 1,
    'filters_in': 32,
    'filters_out': 16,
    'expand_ratio': 1,
    'id_skip': True,
    'strides': 1,
    'se_ratio': 0.25
}, {
    'kernel_size': 3,
    'repeats': 2,
    'filters_in': 16,
    'filters_out': 24,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 5,
    'repeats': 2,
    'filters_in': 24,
    'filters_out': 40,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 3,
    'repeats': 3,
    'filters_in': 40,
    'filters_out': 80,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 5,
    'repeats': 3,
    'filters_in': 80,
    'filters_out': 112,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 1,
    'se_ratio': 0.25
}, {
    'kernel_size': 5,
    'repeats': 4,
    'filters_in': 112,
    'filters_out': 192,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 3,
    'repeats': 1,
    'filters_in': 192,
    'filters_out': 320,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 1,
    'se_ratio': 0.25
}]

CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'truncated_normal'
    }
}

DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}


def EfficientNet(
    width_coefficient,
    depth_coefficient,
    default_size,
    dropout_rate=0.2,
    drop_connect_rate=0.2,
    depth_divisor=8,
    activation='swish',
    blocks_args='default',
    model_name='efficientnet',
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
):
  """Instantiates the EfficientNet architecture using given scaling coefficients.

  Optionally loads weights pre-trained on ImageNet.
  Note that the data format convention used by the model is
  the one specified in your Keras config at `~/.keras/keras.json`.

  Arguments:
    width_coefficient: float, scaling coefficient for network width.
    depth_coefficient: float, scaling coefficient for network depth.
    default_size: integer, default input image size.
    dropout_rate: float, dropout rate before final classifier layer.
    drop_connect_rate: float, dropout rate at skip connections.
    depth_divisor: integer, a unit of network width.
    activation: activation function.
    blocks_args: list of dicts, parameters to construct block modules.
    model_name: string, model name.
    include_top: whether to include the fully-connected
        layer at the top of the network.
    weights: one of `None` (random initialization),
          'imagenet' (pre-training on ImageNet),
          or the path to the weights file to be loaded.
    input_tensor: optional Keras tensor
        (i.e. output of `layers.Input()`)
        to use as image input for the model.
    input_shape: optional shape tuple, only to be specified
        if `include_top` is False.
        It should have exactly 3 inputs channels.
    pooling: optional pooling mode for feature extraction
        when `include_top` is `False`.
        - `None` means that the output of the model will be
            the 4D tensor output of the
            last convolutional layer.
        - `avg` means that global average pooling
            will be applied to the output of the
            last convolutional layer, and thus
            the output of the model will be a 2D tensor.
        - `max` means that global max pooling will
            be applied.
    classes: optional number of classes to classify images
        into, only to be specified if `include_top` is True, and
        if no `weights` argument is specified.
    classifier_activation: A `str` or callable. The activation function to use
        on the "top" layer. Ignored unless `include_top=True`. Set
        `classifier_activation=None` to return the logits of the "top" layer.

  Returns:
    A `keras.Model` instance.

  Raises:
    ValueError: in case of invalid argument for `weights`,
      or invalid input shape.
    ValueError: if `classifier_activation` is not `softmax` or `None` when
      using a pretrained top layer.
  """
  if blocks_args == 'default':
    blocks_args = DEFAULT_BLOCKS_ARGS

  if not (weights in {'imagenet', None} or os.path.exists(weights)):
    raise ValueError('The `weights` argument should be either '
                     '`None` (random initialization), `imagenet` '
                     '(pre-training on ImageNet), '
                     'or the path to the weights file to be loaded.')

  if weights == 'imagenet' and include_top and classes != 1000:
    raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                     ' as true, `classes` should be 1000')

  # Determine proper input shape
  input_shape = imagenet_utils.obtain_input_shape(
      input_shape,
      default_size=default_size,
      min_size=32,
      data_format=backend.image_data_format(),
      require_flatten=include_top,
      weights=weights)

  if input_tensor is None:
    img_input = layers.Input(shape=input_shape)
  else:
    if not backend.is_keras_tensor(input_tensor):
      img_input = layers.Input(tensor=input_tensor, shape=input_shape)
    else:
      img_input = input_tensor

  bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

  def round_filters(filters, divisor=depth_divisor):
    """Round number of filters based on depth multiplier."""
    filters *= width_coefficient
    new_filters = max(divisor, int(filters + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_filters < 0.9 * filters:
      new_filters += divisor
    return int(new_filters)

  def round_repeats(repeats):
    """Round number of repeats based on depth multiplier."""
    return int(math.ceil(depth_coefficient * repeats))

  # Build stem
  x = img_input
  #x = layers.Rescaling(1. / 255.)(x)
  x = layers.Normalization(axis=bn_axis)(x)

  x = layers.ZeroPadding2D(
      padding=imagenet_utils.correct_pad(x, 3),
      name='stem_conv_pad')(x)
  x = layers.Conv2D(
      round_filters(32),
      3,
      strides=2,
      padding='valid',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name='stem_conv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
  x = layers.Activation(activation, name='stem_activation')(x)

  # Build blocks
  blocks_args = copy.deepcopy(blocks_args)

  b = 0
  blocks = float(sum(args['repeats'] for args in blocks_args))
  for (i, args) in enumerate(blocks_args):
    assert args['repeats'] > 0
    # Update block input and output filters based on depth multiplier.
    args['filters_in'] = round_filters(args['filters_in'])
    args['filters_out'] = round_filters(args['filters_out'])

    for j in range(round_repeats(args.pop('repeats'))):
      # The first block needs to take care of stride and filter size increase.
      if j > 0:
        args['strides'] = 1
        args['filters_in'] = args['filters_out']
      x = block(
          x,
          activation,
          drop_connect_rate * b / blocks,
          name='block{}{}_'.format(i + 1, chr(j + 97)),
          **args)
      b += 1

  # Build top
  x = layers.Conv2D(
      round_filters(1280),
      1,
      padding='same',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name='top_conv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name='top_bn')(x)
  x = layers.Activation(activation, name='top_activation')(x)
  if include_top:
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    if dropout_rate > 0:
      x = layers.Dropout(dropout_rate, name='top_dropout')(x)
    imagenet_utils.validate_activation(classifier_activation, weights)
    x = layers.Dense(
        classes,
        activation=classifier_activation,
        kernel_initializer=DENSE_KERNEL_INITIALIZER,
        name='predictions')(x)
  else:
    if pooling == 'avg':
      x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    elif pooling == 'max':
      x = layers.GlobalMaxPooling2D(name='max_pool')(x)

  # Ensure that the model takes into account
  # any potential predecessors of `input_tensor`.
  if input_tensor is not None:
    inputs = layer_utils.get_source_inputs(input_tensor)
  else:
    inputs = img_input

  # Create model.
  model = training.Model(inputs, x, name=model_name)

  # Load weights.
  if weights == 'imagenet':
    if include_top:
      file_suffix = '.h5'
      file_hash = WEIGHTS_HASHES[model_name[-2:]][0]
    else:
      file_suffix = '_notop.h5'
      file_hash = WEIGHTS_HASHES[model_name[-2:]][1]
    file_name = model_name + file_suffix
    weights_path = data_utils.get_file(
        file_name,
        BASE_WEIGHTS_PATH + file_name,
        cache_subdir='models',
        file_hash=file_hash)
    model.load_weights(weights_path)
  elif weights is not None:
    model.load_weights(weights)
  return model


def block(inputs,
          activation='swish',
          drop_rate=0.,
          name='',
          filters_in=32,
          filters_out=16,
          kernel_size=3,
          strides=1,
          expand_ratio=1,
          se_ratio=0.,
          id_skip=True):
  """An inverted residual block.

  Arguments:
      inputs: input tensor.
      activation: activation function.
      drop_rate: float between 0 and 1, fraction of the input units to drop.
      name: string, block label.
      filters_in: integer, the number of input filters.
      filters_out: integer, the number of output filters.
      kernel_size: integer, the dimension of the convolution window.
      strides: integer, the stride of the convolution.
      expand_ratio: integer, scaling coefficient for the input filters.
      se_ratio: float between 0 and 1, fraction to squeeze the input filters.
      id_skip: boolean.

  Returns:
      output tensor for the block.
  """
  bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

  # Expansion phase
  filters = filters_in * expand_ratio
  if expand_ratio != 1:
    x = layers.Conv2D(
        filters,
        1,
        padding='same',
        use_bias=False,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + 'expand_conv')(
            inputs)
    x = layers.BatchNormalization(axis=bn_axis, name=name + 'expand_bn')(x)
    x = layers.Activation(activation, name=name + 'expand_activation')(x)
  else:
    x = inputs

  # Depthwise Convolution
  if strides == 2:
    x = layers.ZeroPadding2D(
        padding=imagenet_utils.correct_pad(x, kernel_size),
        name=name + 'dwconv_pad')(x)
    conv_pad = 'valid'
  else:
    conv_pad = 'same'
  x = layers.DepthwiseConv2D(
      kernel_size,
      strides=strides,
      padding=conv_pad,
      use_bias=False,
      depthwise_initializer=CONV_KERNEL_INITIALIZER,
      name=name + 'dwconv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=name + 'bn')(x)
  x = layers.Activation(activation, name=name + 'activation')(x)

  # Squeeze and Excitation phase
  if 0 < se_ratio <= 1:
    filters_se = max(1, int(filters_in * se_ratio))
    se = layers.GlobalAveragePooling2D(name=name + 'se_squeeze')(x)
    se = layers.Reshape((1, 1, filters), name=name + 'se_reshape')(se)
    se = layers.Conv2D(
        filters_se,
        1,
        padding='same',
        activation=activation,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + 'se_reduce')(
            se)
    se = layers.Conv2D(
        filters,
        1,
        padding='same',
        activation='sigmoid',
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + 'se_expand')(se)
    x = layers.multiply([x, se], name=name + 'se_excite')

  # Output phase
  x = layers.Conv2D(
      filters_out,
      1,
      padding='same',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name=name + 'project_conv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=name + 'project_bn')(x)
  if id_skip and strides == 1 and filters_in == filters_out:
    if drop_rate > 0:
      x = layers.Dropout(
          drop_rate, noise_shape=(None, 1, 1, 1), name=name + 'drop')(x)
    x = layers.add([x, inputs], name=name + 'add')
  return x


@keras_export('keras.applications.efficientnet.EfficientNetB0',
              'keras.applications.EfficientNetB0')
def EfficientNetB0(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.0,
      1.0,
      224,
      0.2,
      model_name='efficientnetb0',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB1',
              'keras.applications.EfficientNetB1')
def EfficientNetB1(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.0,
      1.1,
      240,
      0.2,
      model_name='efficientnetb1',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB2',
              'keras.applications.EfficientNetB2')
def EfficientNetB2(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.1,
      1.2,
      260,
      0.3,
      model_name='efficientnetb2',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB3',
              'keras.applications.EfficientNetB3')
def EfficientNetB3(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.2,
      1.4,
      300,
      0.3,
      model_name='efficientnetb3',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB4',
              'keras.applications.EfficientNetB4')
def EfficientNetB4(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.4,
      1.8,
      380,
      0.4,
      model_name='efficientnetb4',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB5',
              'keras.applications.EfficientNetB5')
def EfficientNetB5(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.6,
      2.2,
      456,
      0.4,
      model_name='efficientnetb5',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB6',
              'keras.applications.EfficientNetB6')
def EfficientNetB6(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      1.8,
      2.6,
      528,
      0.5,
      model_name='efficientnetb6',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.EfficientNetB7',
              'keras.applications.EfficientNetB7')
def EfficientNetB7(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
  return EfficientNet(
      2.0,
      3.1,
      600,
      0.5,
      model_name='efficientnetb7',
      include_top=include_top,
      weights=weights,
      input_tensor=input_tensor,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


@keras_export('keras.applications.efficientnet.preprocess_input')
def preprocess_input(x, data_format=None):  # pylint: disable=unused-argument
  return x


@keras_export('keras.applications.efficientnet.decode_predictions')
def decode_predictions(preds, top=5):
  """Decodes the prediction result from the model.

  Arguments
    preds: Numpy tensor encoding a batch of predictions.
    top: Integer, how many top-guesses to return.

  Returns
    A list of lists of top class prediction tuples
    `(class_name, class_description, score)`.
    One list of tuples per sample in batch input.

  Raises
    ValueError: In case of invalid shape of the `preds` array (must be 2D).
  """
  return imagenet_utils.decode_predictions(preds, top=top)

## Transferência de Aprendizagem e Definição do Modelo.
A transferência de aprendizagem é uma tarefa extremamente comum para Deep Learning. A transferência de aprendizagem é quando um modelo treinado em uma tarefa específica é reutilizada de alguma forma para uma outra tarefa. Os benefícios são a rapidez de treinamento, e até a melhora da acurácia, pois alguns modelos e tarefas podem ser complexos demais para nosso conjunto de dados que muitas vezes são pequenas. A transferência de aprendizagem ajuda dar um pontapé no treinamento neste caso. Sabemos que cada camada aprende características de outputs de camadas de níveis mais baixas. Em geral, camadas mais baixas aprendem características generalizadas que aplicam a um número abrangente de tarefas, como por exemplo, a procura de linhas em imagens. A transferência de aprendizagem funciona melhor para tarefas com características dos dados que foram usados para o treinamento forem gerais. Para mais informações sobre, leia [este artigo](https://machinelearningmastery.com/transfer-learning-for-deep-learning/). Em geral, para usar um modelo pré treinado para sua tarefa, retira-se a camada de classificação do modelo anterior e adiciona outras camadas não treinadas para aprender em cima das características das camadas já treinadas.

O modelo usado para esta tarefa foi o EfficientNetB7. Usamos um modleo pré-treinado no conjunto de dados imagenet, famoso por ser usado como benchmark para modelos de classificação de imagens. 


In [ ]:
#Model creation. Uses EfficientB6 with global average pooling.
with strategy.scope():

    def create_model_eff():
        base = EfficientNetB7(weights='imagenet',input_shape=image_shape,include_top=False)
        inputs = keras.layers.Input(shape=image_shape)
        x = base(inputs)
        x = keras.layers.GlobalAveragePooling2D()(x)
        outputs = keras.layers.Dense(5, activation='softmax')(x)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model

with strategy.scope():
    model = create_model_eff()

Como a camada de output não é inicialmente pré-treinado, é inicialmente perigoso treinar o modelo, pois pode mudar de forma drástica as camadas mais baixas, perdendo grande parte do progresso nos pesos. Há duas alternativas para resolver este problema: Um é congelar todas as camadas do modelo prétreinado e adicionar camadas não treinadas no topo do modelo, e outro é começar com uma taxa de aprendizagem bem baixa para que todas as camadas consigam aprender com a tarefa sem a destruíção dos pesos obtidos. O ultimo será o caso deste modelo. Será aumentado de forma linear até uma certa época, depois será mantida em uma certa taxa, e em seguida diminuída de forma gradativa para o auxílio da convergência do modelo.

In [ ]:
def get_lr_callback(batch_size=BATCH_SIZE):
    lr_start   = 0.000005
    lr_max     = 0.00000125 * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.9
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

## Bagging e Treinamento
[Bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating) se refere ao método de criar vários modelos em que o conjunto de dados para o treinamento utilizado para cada um deles é um subconjunto de dados diferente para cada modelo. Desta forma, os modelos aprendem pesos diferentes e as previsões podem ser mais estáveis. Extremamente útil para conjunto de dados com uma variança alta e geralmente tem uma melhora de performance em comparação ao versão de modelo único, porém é computacionalmente mais caro. A versão de bagging usado para este notebook criará 5 modelos, com separação de dados similar ao separação de dados usado em validação cruzada (Cross Validation). A previsão se dará usando a média das previsões de cada modelo e escolhido a classe com a maior pontuação.

In [ ]:
import gc

#Cria 5 modelos e treina em 5 diferentes folds. Será usado métodos ensemble para a previsão com estes modelos.
def CV_models(model_factory,file_paths,n_epochs,n_steps, cv=5 ):
    
    print('The current working directory is: ', os.getcwd())
    
    
    folds = KFold(n_splits=cv)
    histories = []
    
    for i, (train_idx, test_idx) in enumerate(folds.split(file_paths)):
        
        #Defining training and testing datasets
        training_files = file_paths[train_idx]
        test_files = file_paths[test_idx]
        train_ds = training_dataset(training_files)
        test_ds = validation_dataset(test_files)
        
        #Defining the model file name
        model_file_name = 'effiecint_b5_fold_{}.hdf5'.format(i)
        
        #Utilizamos vários callbacks para o auxílio do treinamento. EarlyStopping para o treinamento quando o loss converge. ModelChecpoint salva o modelo caso algo acontecer e 
        #você não perder o modelo.
        
        early_stopping = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
        checkpoint = keras.callbacks.ModelCheckpoint(model_file_name,save_best_only=True,monitor='val_loss',\
                                                     verbose=1)
        callbacks = [early_stopping,checkpoint, get_lr_callback()]
        
        
        
        tf.keras.backend.clear_session() 
        with strategy.scope():
            
            loss = keras.losses.CategoricalCrossentropy(label_smoothing=0.001)
            #Um otimizador interessante é o RAdam, ou rectified Adam. É melhor que o Adam, pois é robusto a diferentes taxas de aprendizagem.
            opt=RAdam()
            
            potential_model = Path('/kaggle/input') / 'modelss7' / model_file_name
            
            #If model exists in input, 
            if potential_model.exists():
                print("{} exists".format(potential_model))
                continue
                #current_model = tf.keras.models.load_model(potential_model)
            else:
                current_model = model_factory() 
                
            current_model.compile(loss=loss, optimizer=opt, metrics=['categorical_accuracy'])
            
        
        history=current_model.fit(train_ds, epochs=n_epochs, steps_per_epoch=n_steps, callbacks=callbacks, validation_data=test_ds)
        
        
        del current_model
        del train_ds
        del test_ds
        z = gc.collect()
        histories.append(history)
            
        
            
        
    return histories


In [ ]:
history = CV_models(create_model_eff, np.array(files),EPOCHS,steps_per_epoch)